In [103]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [104]:
cd ../

c:\Users\okpl8\Documents


In [105]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data/train.csv'

In [ ]:
train = train.drop('ID', axis = 1)
test = test.drop('ID', axis = 1)

In [ ]:
train.drop(columns=['PGD 시술 여부','PGS 시술 여부','난자 해동 경과일','배아 해동 경과일',
            '임신 시도 또는 마지막 임신 경과 연수','착상 전 유전 검사 사용 여부'],inplace=True)
test.drop(columns=['PGD 시술 여부','PGS 시술 여부','난자 해동 경과일','배아 해동 경과일',
            '임신 시도 또는 마지막 임신 경과 연수','착상 전 유전 검사 사용 여부'],inplace=True)

In [ ]:
train["특정 시술 유형"] = train["특정 시술 유형"].fillna("Unknown").astype(str)
test["특정 시술 유형"] = test["특정 시술 유형"].fillna("Unknown").astype(str)

def categorize_treatment(x):
    first_part = x.split(":")[0]  # 첫 번째 값 가져오기

    if first_part == "ICSI":
        return "ICSI"
    elif first_part == "IVF":
        return "IVF"
    elif first_part in ["IUI", "ICI"]:
        return "IUI"
    else:
        return "Unknown"

# ✅ 그룹화 적용 (train & test)
train["특정 시술 유형"] = train["특정 시술 유형"].apply(categorize_treatment)
test["특정 시술 유형"] = test["특정 시술 유형"].apply(categorize_treatment)

In [ ]:
mode_cols = ["착상 전 유전 진단 사용 여부", "배아 생성 주요 이유", "해동 난자 수", "저장된 신선 난자 수", "기증자 정자와 혼합된 난자 수",
             "기증 배아 사용 여부", "대리모 여부"]

for col in mode_cols:
    if col in train.columns and col in test.columns:  # train과 test 모두 해당 컬럼이 있을 때만 실행
        mean_value = train[col].mode()[0]  # 🚨 Train 데이터의 최빈값
        train[col] = train[col].fillna(mean_value)  # inplace=False 방식으로 수정
        test[col] = test[col].fillna(mean_value)    # inplace=False 방식으로 수정
    else:
        print(f"컬럼 {col}이(가) train 또는 test 데이터에 없습니다.")

In [ ]:
mode_cols = ["단일 배아 이식 여부", "미세주입 후 저장된 배아 수", "해동된 배아 수", "동결 배아 사용 여부", "신선 배아 사용 여부",
             "난자 채취 경과일", "난자 혼합 경과일","저장된 배아 수","미세주입에서 생성된 배아 수","미세주입 배아 이식 수", "미세주입된 난자 수"]

for col in mode_cols:
    if col in train.columns and col in test.columns:  # train과 test 모두 해당 컬럼이 있을 때만 실행
        mean_value = train[col].mode()[0]  # 🚨 Train 데이터의 최빈값
        train[col] = train[col].fillna(mean_value)  # inplace=False 방식으로 수정
        test[col] = test[col].fillna(mean_value)    # inplace=False 방식으로 수정
    else:
        print(f"컬럼 {col}이(가) train 또는 test 데이터에 없습니다.")

In [ ]:
median_cols = ['총 생성 배아 수','','이식된 배아 수','수집된 신선 난자 수','혼합된 난자 수','파트너 정자와 혼합된 난자 수','배아 이식 경과일']
for col in median_cols:
    if col in train.columns and col in test.columns:  # train과 test 모두 해당 컬럼이 있을 때만 실행
        mean_value = train[col].median()  # 🚨 Train 데이터의 중앙값
        train[col] = train[col].fillna(mean_value)  # inplace=False 방식으로 수정
        test[col] = test[col].fillna(mean_value)    # inplace=False 방식으로 수정
    else:
        print(f"컬럼 {col}이(가) train 또는 test 데이터에 없습니다.")

컬럼 이(가) train 또는 test 데이터에 없습니다.


In [ ]:
train.drop(columns=['배란 유도 유형','난자 기증자 나이','정자 기증자 나이'],inplace=True)
test.drop(columns=['배란 유도 유형','난자 기증자 나이','정자 기증자 나이'],inplace=True)

In [ ]:
train.drop(columns=['난자 채취 경과일','불임 원인 - 여성 요인','불임 원인 - 정자 면역학적 요인'],inplace=True)
test.drop(columns=['난자 채취 경과일','불임 원인 - 여성 요인','불임 원인 - 정자 면역학적 요인'],inplace=True)

In [ ]:

# 배아 생성 주요 이유 그룹화 함수
def categorize_embryo_reason(x):
    if "현재 시술용" in x:
        return "Current"
    elif "배아 저장용" in x:
        return "Storage"
    elif "기증용" in x:
        return "Donation"
    elif "난자 저장용" in x:
        return "Egg Storage"
    elif "연구용" in x:
        return "Research"
    else:
        return "Unknown"

train["배아 생성 주요 이유"] = train["배아 생성 주요 이유"].apply(categorize_embryo_reason)
test["배아 생성 주요 이유"] = test["배아 생성 주요 이유"].apply(categorize_embryo_reason)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_col=['시술 시기 코드','시술 당시 나이','총 시술 횟수','클리닉 내 총 시술 횟수','IVF 시술 횟수','DI 시술 횟수',
           '총 임신 횟수','IVF 임신 횟수','DI 임신 횟수','총 출산 횟수','IVF 출산 횟수','DI 출산 횟수']
# Label Encoding

for col in label_col:
    # Label Encoder 생성
    le = LabelEncoder()
    
    # Train 데이터에 Label Encoding 적용
    train[col] = le.fit_transform(train[col].astype(str))
    
    # Test 데이터에 동일한 Label Encoding 적용
    test[col] = le.transform(test[col].astype(str))  # Test에는 transform만 적용

# 인코딩 확인
print(train[label_col].head())
print(test[label_col].head())

   시술 시기 코드  시술 당시 나이  총 시술 횟수  클리닉 내 총 시술 횟수  IVF 시술 횟수  DI 시술 횟수  총 임신 횟수  \
0         6         0        0              0          0         0        0   
1         5         5        0              0          0         0        0   
2         3         0        1              0          1         0        0   
3         2         1        1              1          1         0        0   
4         3         0        0              0          0         0        0   

   IVF 임신 횟수  DI 임신 횟수  총 출산 횟수  IVF 출산 횟수  DI 출산 횟수  
0          0         0        0          0         0  
1          0         0        0          0         0  
2          0         0        0          0         0  
3          0         0        0          0         0  
4          0         0        0          0         0  
   시술 시기 코드  시술 당시 나이  총 시술 횟수  클리닉 내 총 시술 횟수  IVF 시술 횟수  DI 시술 횟수  총 임신 횟수  \
0         5         1        1              1          1         0        0   
1         1         0        0       

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot_col=['시술 유형','특정 시술 유형','배아 생성 주요 이유','난자 출처','정자 출처'] 

# OneHotEncoder 생성
ohe = OneHotEncoder(sparse_output=False, drop='first')  # drop='first'는 더미 변수 함정 방지

# Train 데이터에 One-Hot Encoding 적용
train_encoded = ohe.fit_transform(train[onehot_col])

# Test 데이터에 동일한 One-Hot Encoding 적용
test_encoded = ohe.transform(test[onehot_col])

# 인코딩된 데이터를 DataFrame으로 변환하고 기존 데이터와 합치기
train_encoded_df = pd.DataFrame(train_encoded, columns=ohe.get_feature_names_out(onehot_col))
test_encoded_df = pd.DataFrame(test_encoded, columns=ohe.get_feature_names_out(onehot_col))

train = pd.concat([train.drop(columns=onehot_col), train_encoded_df], axis=1)
test = pd.concat([test.drop(columns=onehot_col), test_encoded_df], axis=1)

In [ ]:
train.drop(columns=['남성 주 불임 원인','IVF 시술 횟수','IVF 임신 횟수','IVF 출산 횟수','미세주입에서 생성된 배아 수',
                    '파트너 정자와 혼합된 난자 수'],inplace=True)
test.drop(columns=['남성 주 불임 원인','IVF 시술 횟수','IVF 임신 횟수','IVF 출산 횟수','미세주입에서 생성된 배아 수',
                    '파트너 정자와 혼합된 난자 수'],inplace=True)

In [ ]:
train.drop(columns=['여성 주 불임 원인','DI 임신 횟수','DI 출산 횟수','혼합된 난자 수'],inplace=True)
test.drop(columns=['여성 주 불임 원인','DI 임신 횟수','DI 출산 횟수','혼합된 난자 수'],inplace=True)

In [ ]:
train.drop(columns=['부부 부 불임 원인','총 시술 횟수','총 출산 횟수','수집된 신선 난자 수','동결 배아 사용 여부',
                    '신선 배아 사용 여부'],inplace=True)
test.drop(columns=['부부 부 불임 원인','총 시술 횟수','총 출산 횟수','수집된 신선 난자 수','동결 배아 사용 여부',
                    '신선 배아 사용 여부'],inplace=True)

In [ ]:
# 상관행렬 계산
corr_matrix = train.corr()

# 상관관계가 0.7 이상인 값만 선택 (자기 자신과의 상관관계 제외)
high_corr = corr_matrix[(corr_matrix > 0.7) & (corr_matrix < 1)]

# 결과를 변수와 해당 변수와 0.7 이상인 상관관계를 가진 다른 변수들로 출력
for col in high_corr.columns:
    related_vars = high_corr[col].dropna()  # 결측치 제외
    related_vars = related_vars[related_vars >= 0.7]  # 0.7 이상인 값만 선택
    if len(related_vars) > 0:  # 상관관계가 0.7 이상인 값이 있을 경우
        print(f"{col}: {', '.join([f'{var}: {related_vars[var]:.2f}' for var in related_vars.index])}")

특정 시술 유형_IUI: 난자 출처_알 수 없음: 0.99
난자 출처_알 수 없음: 특정 시술 유형_IUI: 0.99


In [ ]:
mode_cols = ["단일 배아 이식 여부", "미세주입 후 저장된 배아 수", "해동된 배아 수", "동결 배아 사용 여부", "신선 배아 사용 여부",
             "난자 채취 경과일", "난자 혼합 경과일","저장된 배아 수","미세주입에서 생성된 배아 수","미세주입 배아 이식 수", "미세주입된 난자 수"]

for col in mode_cols:
    if col in train.columns and col in test.columns:  # train과 test 모두 해당 컬럼이 있을 때만 실행
        mean_value = train[col].mode()[0]  # 🚨 Train 데이터의 최빈값
        train[col] = train[col].fillna(mean_value)  # inplace=False 방식으로 수정
        test[col] = test[col].fillna(mean_value)    # inplace=False 방식으로 수정
    else:
        print(f"컬럼 {col}이(가) train 또는 test 데이터에 없습니다.")

컬럼 동결 배아 사용 여부이(가) train 또는 test 데이터에 없습니다.
컬럼 신선 배아 사용 여부이(가) train 또는 test 데이터에 없습니다.
컬럼 난자 채취 경과일이(가) train 또는 test 데이터에 없습니다.
컬럼 미세주입에서 생성된 배아 수이(가) train 또는 test 데이터에 없습니다.


In [ ]:
mode_cols = ["착상 전 유전 진단 사용 여부", "배아 생성 주요 이유", "해동 난자 수", "저장된 신선 난자 수", "기증자 정자와 혼합된 난자 수",
             "기증 배아 사용 여부", "대리모 여부"]

for col in mode_cols:
    if col in train.columns and col in test.columns:  # train과 test 모두 해당 컬럼이 있을 때만 실행
        mean_value = train[col].mode()[0]  # 🚨 Train 데이터의 최빈값
        train[col] = train[col].fillna(mean_value)  # inplace=False 방식으로 수정
        test[col] = test[col].fillna(mean_value)    # inplace=False 방식으로 수정
    else:
        print(f"컬럼 {col}이(가) train 또는 test 데이터에 없습니다.")

컬럼 배아 생성 주요 이유이(가) train 또는 test 데이터에 없습니다.


-------------shap-------------

In [ ]:
drop_features = [
    "배아 생성 주요 이유_Donation", "배아 생성 주요 이유_Egg Storage", "불임 원인 - 자궁경부 문제",
    "불임 원인 - 정자 운동성", "특정 시술 유형_IUI", "난자 혼합 경과일", "해동 난자 수",
    "대리모 여부", "불임 원인 - 정자 형태"
]
train.drop(columns=drop_features, inplace=True)
test.drop(columns=drop_features, inplace=True)

-------

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import numpy as np

# ✅ 데이터 분할
X = train.drop(columns=['임신 성공 여부'])
y = train['임신 성공 여부']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

----ml----

In [ ]:
models = {
    "Extra Trees": ExtraTreesClassifier(max_depth= 15, min_samples_split = 10, n_estimators = 619, random_state=42, n_jobs=-1),
    "CatBoost": CatBoostClassifier(depth = 5, iterations= 4733, learning_rate= 0.052153844058674126, verbose=0, random_state=42),
    "LightGBM": LGBMClassifier(max_depth=4, learning_rate=  0.04837287275958974, n_estimators= 472, random_state=42, n_jobs=-1),
    'XGBoost': XGBClassifier(max_depth = 4, learning_rate= 0.02379326289805242, n_estimators=917, random_state=42, n_jobs=-1),
    "Random Forest": RandomForestClassifier(max_depth=15, n_estimators=494, min_samples_split=5, random_state=42),
}

In [ ]:
from sklearn.calibration import calibration_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import matplotlib.font_manager as fm

from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_curve, auc, precision_recall_curve, confusion_matrix,
                           classification_report, average_precision_score)
from sklearn.calibration import calibration_curve 
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import VotingClassifier

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
#skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

# ✅ 각 모델별 Test 예측값을 저장할 리스트 (Soft Voting 적용을 위한 확률 평균)
test_predictions = {name: np.zeros(X_test.shape[0]) for name in models.keys() if hasattr(models[name], "predict_proba")}
submission_predictions = {name: np.zeros(test.shape[0]) for name in models.keys() if hasattr(models[name], "predict_proba")}

# ✅ KFold 적용
roc_auc_scores = []  # ROC-AUC 점수 저장 리스트

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\n=== Fold {fold+1} ===")

    # KFold 데이터 분할
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    Y_train_fold, Y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    for name, model in models.items():
        if hasattr(model, "predict_proba"):  # `predict_proba()`를 지원하는 모델만 사용
            print(f"\n--- Training {name} ---")
            
            # 모델 학습
            model.fit(X_train_fold, Y_train_fold)
            # 훈련 세트 성능 평가
            Y_train_pred = model.predict(X_train_fold)
            print("\n--- Training Set Performance ---")
            print(classification_report(Y_train_fold, Y_train_pred))
            

            # ✅ 검증 데이터 예측 (Positive Class 확률만 저장)
            Y_val_pred_proba = model.predict_proba(X_val_fold)[:, 1]
            # 검증 세트 성능 평가
            Y_val_pred = model.predict(X_val_fold)
            print("\n--- Validation Set Performance ---")
            print(classification_report(Y_val_fold, Y_val_pred))

            # ✅ ROC-AUC 점수 계산 및 저장
            roc_auc = roc_auc_score(Y_val_fold, Y_val_pred_proba)
            roc_auc_scores.append(roc_auc)
            print(f"📊 ROC-AUC (Fold {fold+1}, {name}): {roc_auc:.4f}")

            # ✅ X_test 예측값 저장 (Soft Voting을 위해 확률 예측값 사용)
            test_pred = model.predict_proba(X_test)[:, 1]  # Positive Class(1) 확률
            test_predictions[name] += test_pred / kf.n_splits  # 확률 평균 계산

            # ✅ 실제 test.csv 데이터 예측값 저장
            submission_pred = model.predict_proba(test)[:, 1]  # Positive Class(1) 확률
            submission_predictions[name] += submission_pred / kf.n_splits  # 확률 평균 계산

        else:
            print(f"⚠ Warning: {name} 모델은 `predict_proba()`를 지원하지 않습니다. 제외됨.")

# ✅ Platt Scaling (Sigmoid) 적용
calibrated_models = {}

for model_name in ["Random Forest", "CatBoost", "Extra Trees", "LightGBM"]:
    try:
        print(f"Calibrating {model_name}...")
        calibrated_models[model_name] = CalibratedClassifierCV(models[model_name], method="isotonic", cv=5)
        calibrated_models[model_name].fit(X_train, y_train)
    except Exception as e:
        print(f"⚠️ Calibration failed for {model_name}: {e}")

# ✅ 보정된 확률 예측 저장
calibrated_pred_proba_test = {
    model_name: calibrated_models[model_name].predict_proba(X_test)[:, 1]
    for model_name in calibrated_models
}

calibrated_pred_proba_submission = {
    model_name: calibrated_models[model_name].predict_proba(test)[:, 1]
    for model_name in calibrated_models
}

# ✅ 최적 가중치 적용 후 Soft Voting
ensemble_pred_proba_test = (
    test_predictions["XGBoost"] * 0.41590671750912966 +
    calibrated_pred_proba_test["LightGBM"] * 0.4985557677497247 +
    calibrated_pred_proba_test["CatBoost"] * 0.17378238941331203 +
    calibrated_pred_proba_test["Random Forest"] * 0.05236890594772381 +
    calibrated_pred_proba_test["Extra Trees"] * 0.05283185785986821
)

ensemble_pred_proba_submission = (
    submission_predictions["XGBoost"] * 0.41590671750912966 +
    calibrated_pred_proba_submission["LightGBM"] * 0.4985557677497247 +
    calibrated_pred_proba_submission["CatBoost"] * 0.17378238941331203 +
    calibrated_pred_proba_submission["Random Forest"] *0.05236890594772381 +
    calibrated_pred_proba_submission["Extra Trees"] * 0.05283185785986821
)


=== Fold 1 ===

--- Training Extra Trees ---

--- Training Set Performance ---
              precision    recall  f1-score   support

           0       0.75      1.00      0.85    121470
           1       0.93      0.03      0.06     42594

    accuracy                           0.75    164064
   macro avg       0.84      0.52      0.46    164064
weighted avg       0.79      0.75      0.65    164064


--- Validation Set Performance ---
              precision    recall  f1-score   support

           0       0.75      1.00      0.85     30508
           1       0.55      0.01      0.02     10508

    accuracy                           0.74     41016
   macro avg       0.65      0.50      0.44     41016
weighted avg       0.70      0.74      0.64     41016

📊 ROC-AUC (Fold 1, Extra Trees): 0.7340

--- Training CatBoost ---

--- Training Set Performance ---
              precision    recall  f1-score   support

           0       0.77      0.96      0.86    121470
           1       0

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.metrics import (roc_auc_score, classification_report, 
                             precision_recall_curve, roc_curve)


print(torch.cuda.is_available())

True


In [ ]:
print(torch.cuda.is_available())  # True가 나오면 GPU 사용 가능
print(torch.cuda.get_device_name(0))  # 사용 중인 GPU 이름 출력

True
NVIDIA GeForce RTX 4060


In [ ]:
X_train.shape

(205080, 37)

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, precision_recall_curve, roc_curve

###########################################
# 1. Focal Loss 구현
###########################################
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, inputs, targets):
        # inputs: 예측값 (로짓 값; sigmoid 적용 전)
        # targets: 정답 (0 또는 1, float tensor)
        inputs = inputs.clamp(min=1e-7, max=1-1e-7)  # log(0) 방지
        BCE_loss = F.binary_cross_entropy(torch.sigmoid(inputs), targets, reduction='none')
        pt = torch.where(targets == 1, torch.sigmoid(inputs), 1 - torch.sigmoid(inputs))
        loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

focal_loss = FocalLoss(alpha=0.25, gamma=2.0, reduction='mean')

###########################################
# 2. Custom TabNetClassifier with Focal Loss
###########################################
class CustomTabNetClassifier(TabNetClassifier):
    """
    TabNetClassifier를 상속하여 focal loss를 사용하도록 _get_loss() 메서드를 오버라이드합니다.
    """
    def __init__(self, focal_loss, **kwargs):
        super(CustomTabNetClassifier, self).__init__(**kwargs)
        self.focal_loss = focal_loss

    def _get_loss(self, preds, targets, weights=None):
        # preds: 모델의 로짓 출력, targets: 정답 (float tensor)
        return self.focal_loss(preds, targets)

###########################################
# 3. 데이터 준비 (TabNet은 NumPy array 입력)
###########################################
# X_train, y_train, X_test, test는 이미 전처리된 DataFrame이라고 가정합니다.
X_train_tab = X_train.values.astype(np.float32)
y_train_tab = y_train.values.astype(np.float32)  # y_train
X_test_tab = X_test.values.astype(np.float32)
X_submission_tab = test.values.astype(np.float32)  # 제출용 test 데이터

###########################################
# 4. 5-Fold Stratified CV를 이용한 TabNet 학습 (GPU 사용)
###########################################
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
stacking_train_meta_tab = np.zeros((X_train_tab.shape[0], 1))
stacking_test_meta_tab = np.zeros((X_test_tab.shape[0], 1))
stacking_submission_meta_tab = np.zeros((X_submission_tab.shape[0], 1))  # 제출 예측용

fold_idx = 0
for train_index, val_index in skf.split(X_train_tab, y_train_tab):
    print(f"\n=== TabNet Fold {fold_idx+1} ===")
    X_train_fold_tab, X_val_fold_tab = X_train_tab[train_index], X_train_tab[val_index]
    y_train_fold_tab, y_val_fold_tab = y_train_tab[train_index], y_train_tab[val_index]
    
    # CustomTabNetClassifier 생성 (GPU 사용, focal loss 적용)
    tabnet_clf = CustomTabNetClassifier(
        focal_loss=focal_loss,
        device_name='cuda',  # GPU 사용 명시
        optimizer_params=dict(lr=0.02),
        scheduler_params={"step_size": 10, "gamma": 0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        seed=42,
        verbose=1
    )
    
    # TabNet 학습
    tabnet_clf.fit(
        X_train_fold_tab, y_train_fold_tab,
        eval_set=[(X_val_fold_tab, y_val_fold_tab)],
        eval_metric=['auc'],
        max_epochs=50,
        patience=10,
        batch_size=64,
        virtual_batch_size=32,
        num_workers=0,
        drop_last=False
    )
    
    # 검증 Fold에 대한 예측 (메타 데이터로 사용)
    stacking_train_meta_tab[val_index, 0] = tabnet_clf.predict_proba(X_val_fold_tab)[:, 1]
    
    # X_test에 대한 예측 (각 Fold 예측을 평균)
    stacking_test_meta_tab[:, 0] += tabnet_clf.predict_proba(X_test_tab)[:, 1] / skf.n_splits
    
    # 제출용 test 데이터에 대한 예측 (각 Fold 예측을 평균)
    stacking_submission_meta_tab[:, 0] += tabnet_clf.predict_proba(X_submission_tab)[:, 1] / skf.n_splits
    
    fold_idx += 1

###########################################
# 5. TabNet 성능 평가
###########################################
tabnet_train_roc_auc = roc_auc_score(y_train, stacking_train_meta_tab.ravel())
print(f"\nTabNet Training ROC-AUC: {tabnet_train_roc_auc:.4f}")

tabnet_test_roc_auc = roc_auc_score(y_test, stacking_test_meta_tab.ravel())
print(f"TabNet Test ROC-AUC: {tabnet_test_roc_auc:.4f}")

precision_tab, recall_tab, thresholds_tab = precision_recall_curve(y_test, stacking_test_meta_tab.ravel())
f1_scores_tab = (2 * precision_tab * recall_tab) / (precision_tab + recall_tab + 1e-8)
optimal_threshold_tab = thresholds_tab[np.argmax(f1_scores_tab)]
print(f"TabNet Optimal Threshold: {optimal_threshold_tab:.4f}")

Y_test_pred_tab = (stacking_test_meta_tab.ravel() > optimal_threshold_tab).astype(int)
print("\n=== TabNet Classification Report ===")
print(classification_report(y_test, Y_test_pred_tab))

###########################################
# 6. Kaggle 제출 파일 생성 (TabNet)
###########################################
submission_filename = f'./data/submission_tabnet_focal_{tabnet_test_roc_auc:.4f}.csv'
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['probability'] = stacking_submission_meta_tab.ravel()
sample_submission.to_csv(submission_filename, index=False)
print(f"\n✅ Submission file saved: {submission_filename}")



=== TabNet Fold 1 ===


c:\Users\okpl8\anaconda3\envs\LG2\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.51151 | val_0_auc: 0.72101 |  0:00:41s
epoch 1  | loss: 0.49875 | val_0_auc: 0.72513 |  0:01:24s
epoch 2  | loss: 0.49918 | val_0_auc: 0.7281  |  0:02:05s
epoch 3  | loss: 0.49787 | val_0_auc: 0.73034 |  0:02:47s
epoch 4  | loss: 0.49648 | val_0_auc: 0.73037 |  0:03:28s
epoch 5  | loss: 0.4961  | val_0_auc: 0.72977 |  0:04:14s
epoch 6  | loss: 0.49563 | val_0_auc: 0.73144 |  0:04:55s
epoch 7  | loss: 0.49537 | val_0_auc: 0.73135 |  0:05:37s
epoch 8  | loss: 0.49529 | val_0_auc: 0.73055 |  0:06:18s
epoch 9  | loss: 0.49452 | val_0_auc: 0.73177 |  0:07:01s
epoch 10 | loss: 0.4945  | val_0_auc: 0.73185 |  0:07:43s
epoch 11 | loss: 0.49376 | val_0_auc: 0.73218 |  0:08:25s
epoch 12 | loss: 0.49355 | val_0_auc: 0.73278 |  0:09:07s
epoch 13 | loss: 0.49424 | val_0_auc: 0.73188 |  0:09:47s
epoch 14 | loss: 0.49359 | val_0_auc: 0.73351 |  0:10:36s
epoch 15 | loss: 0.49369 | val_0_auc: 0.7338  |  0:11:17s
epoch 16 | loss: 0.49323 | val_0_auc: 0.73402 |  0:11:58s
epoch 17 | los

c:\Users\okpl8\anaconda3\envs\LG2\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



=== TabNet Fold 2 ===


...


=== TabNet Fold 5 ===


c:\Users\okpl8\anaconda3\envs\LG2\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.51586 | val_0_auc: 0.72075 |  0:00:41s
epoch 1  | loss: 0.50185 | val_0_auc: 0.72303 |  0:01:22s
epoch 2  | loss: 0.49807 | val_0_auc: 0.7318  |  0:02:04s
epoch 3  | loss: 0.49732 | val_0_auc: 0.73179 |  0:02:45s
epoch 4  | loss: 0.496   | val_0_auc: 0.73289 |  0:03:29s
epoch 5  | loss: 0.49527 | val_0_auc: 0.73354 |  0:04:11s
epoch 6  | loss: 0.49502 | val_0_auc: 0.73311 |  0:04:55s
epoch 7  | loss: 0.49486 | val_0_auc: 0.73459 |  0:05:37s
epoch 8  | loss: 0.49438 | val_0_auc: 0.73209 |  0:06:20s
epoch 9  | loss: 0.4946  | val_0_auc: 0.73489 |  0:07:03s
epoch 10 | loss: 0.49413 | val_0_auc: 0.73453 |  0:07:45s
epoch 11 | loss: 0.49429 | val_0_auc: 0.73447 |  0:08:27s
epoch 12 | loss: 0.49339 | val_0_auc: 0.73165 |  0:09:10s
epoch 13 | loss: 0.49352 | val_0_auc: 0.73471 |  0:09:56s
epoch 14 | loss: 0.49372 | val_0_auc: 0.73591 |  0:10:38s
epoch 15 | loss: 0.49311 | val_0_auc: 0.73524 |  0:11:19s
epoch 16 | loss: 0.49327 | val_0_auc: 0.73551 |  0:12:03s
epoch 17 | los

c:\Users\okpl8\anaconda3\envs\LG2\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



TabNet Training ROC-AUC: 0.7346
TabNet Test ROC-AUC: 0.7370
TabNet Optimal Threshold: 0.2680

=== TabNet Classification Report ===
              precision    recall  f1-score   support

           0       0.88      0.57      0.69     38145
           1       0.38      0.77      0.51     13126

    accuracy                           0.62     51271
   macro avg       0.63      0.67      0.60     51271
weighted avg       0.75      0.62      0.65     51271


✅ Submission file saved: ./data/submission_tabnet_focal_0.7370.csv


In [120]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score


# ✅ Tensor 변환
X_train_np = X_train.values.astype(np.float32)
y_train_np = y_train.values.astype(np.float32)  # shape: (n_samples,)
X_test_np = X_test.values.astype(np.float32)
X_submission_np = test.values.astype(np.float32)




# 2. MLP 모델 정의 (출력은 logit으로 계산, loss에 BCEWithLogitsLoss 사용)
class DeepMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_hidden_layers=10):
        super(DeepMLP, self).__init__() 
        layers = []
        # 첫번째 층: 입력 -> hidden_dim
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())
        # 중간 hidden layer: hidden_dim -> hidden_dim (총 num_hidden_layers - 1번 추가)
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        # 마지막 출력층: hidden_dim -> 1
        layers.append(nn.Linear(hidden_dim, 1))
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)  # 로짓 값을 출력

def build_mlp(input_dim):
    return DeepMLP(input_dim=input_dim, hidden_dim=128, num_hidden_layers=10)

# 3. KFold 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_auc_scores = []

# stacking 메타 데이터: train과 test에 대한 예측값 저장
stacking_train_meta = np.zeros((X_train_np.shape[0], 1))
stacking_test_meta = np.zeros((X_test_np.shape[0], 1))
stacking_submission_meta = np.zeros((X_submission_np.shape[0],1))

# 학습 파라미터 설정
num_epochs = 20
batch_size = 256
learning_rate = 0.001


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# 4. KFold Cross Validation
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_np)):
    print(f"\n=== Fold {fold+1} Training ===")
    
    # Fold별 데이터 분할
    X_tr = X_train_np[train_idx]
    y_tr = y_train_np[train_idx]
    X_val = X_train_np[val_idx]
    y_val = y_train_np[val_idx]
    
    # TensorDataset 및 DataLoader 생성
    train_dataset = TensorDataset(torch.tensor(X_tr), torch.tensor(y_tr).unsqueeze(1))
    val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val).unsqueeze(1))
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # 모델 생성
    model = build_mlp(X_train_np.shape[1]).to(device)
    
    # 손실 함수와 옵티마이저 설정 (BCEWithLogitsLoss: 내부적으로 sigmoid 적용)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # 학습 루프
    best_val_auc = 0.0
    best_epoch = 0
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X)  # 로짓 출력
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * batch_X.size(0)
        epoch_loss /= len(train_loader.dataset)
        
        # 매 epoch마다 검증 데이터에 대해 모델 평가
        model.eval()
        val_preds = []
        val_targets = []
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X = batch_X.to(device)
                outputs = model(batch_X)
                preds = torch.sigmoid(outputs).cpu().numpy().flatten()  # 확률로 변환
                val_preds.extend(preds)
                val_targets.extend(batch_y.cpu().numpy().flatten())
        val_auc = roc_auc_score(val_targets, val_preds)
        
        # epoch마다 loss와 validation AUC 출력
        if (epoch+1) % 2 ==0  or (epoch+1) % 3 == 0 or (epoch+1) == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val AUC: {val_auc:.4f}")
        
        # best validation AUC 갱신
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_epoch = epoch+1
            
    fold_auc_scores.append(best_val_auc)
    print(f"Fold {fold+1} best Val AUC: {best_val_auc:.4f} at epoch {best_epoch}")
    
# --- 메타 데이터 저장 ---
    # 검증 데이터에 대한 예측값 저장
    model.eval()
    fold_val_preds = []
    with torch.no_grad():
        for batch_X, _ in val_loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            fold_val_preds.extend(preds)
    stacking_train_meta[val_idx, 0] = np.array(fold_val_preds)
    
    # Test 데이터에 대한 예측 (각 Fold 예측을 평균)
    test_dataset = TensorDataset(torch.tensor(X_test_np))
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    fold_test_preds = []
    with torch.no_grad():
        for batch_X, in test_loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            fold_test_preds.extend(preds)
    stacking_test_meta[:, 0] += np.array(fold_test_preds) / kf.n_splits
    
    # 제출 데이터에 대한 예측 (각 Fold 예측을 평균)
    submission_dataset = TensorDataset(torch.tensor(X_submission_np))
    submission_loader = DataLoader(submission_dataset, batch_size=batch_size, shuffle=False)
    fold_submission_preds = []
    with torch.no_grad():
        for batch_X, in submission_loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            fold_submission_preds.extend(preds)
    stacking_submission_meta[:, 0] += np.array(fold_submission_preds) / kf.n_splits

# -------------------------------
# 평균 ROC-AUC (train 메타 데이터 기준)
# -------------------------------
meta_train_auc = roc_auc_score(y_train_np, stacking_train_meta.ravel())
print(f"\n📈 스태킹 Train ROC-AUC: {meta_train_auc:.4f}")


# Test 데이터에 대한 최종 예측 결과
meta_test_auc = roc_auc_score(y_test, stacking_test_meta.ravel())  # 만약 y_test가 있다면 roc_auc_score를 계산할 수 있습니다.
print("\n앙상블된 Test 예측 결과 (평균):")
print(f"\n📈 스태킹 test ROC-AUC: {meta_test_auc:.4f}")


submission_filename = f'./data/submission_mlp_{meta_test_auc:.4f}.csv'
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['probability'] = stacking_submission_meta.ravel()
sample_submission.to_csv(submission_filename, index=False)
print(f"\n✅ Submission file saved: {submission_filename}")

Using device: cuda

=== Fold 1 Training ===
Epoch 2/20, Loss: 0.4958, Val AUC: 0.7334
Epoch 3/20, Loss: 0.4945, Val AUC: 0.7348
Epoch 4/20, Loss: 0.4932, Val AUC: 0.7356
Epoch 6/20, Loss: 0.4920, Val AUC: 0.7362
Epoch 8/20, Loss: 0.4908, Val AUC: 0.7374
Epoch 9/20, Loss: 0.4906, Val AUC: 0.7366
Epoch 10/20, Loss: 0.4902, Val AUC: 0.7376
Epoch 12/20, Loss: 0.4895, Val AUC: 0.7379
Epoch 14/20, Loss: 0.4887, Val AUC: 0.7373
Epoch 15/20, Loss: 0.4882, Val AUC: 0.7368
Epoch 16/20, Loss: 0.4881, Val AUC: 0.7370
Epoch 18/20, Loss: 0.4873, Val AUC: 0.7325
Epoch 20/20, Loss: 0.4863, Val AUC: 0.7357
Fold 1 best Val AUC: 0.7379 at epoch 12

=== Fold 2 Training ===
Epoch 2/20, Loss: 0.4946, Val AUC: 0.7329
Epoch 3/20, Loss: 0.4929, Val AUC: 0.7330
Epoch 4/20, Loss: 0.4922, Val AUC: 0.7340
Epoch 6/20, Loss: 0.4907, Val AUC: 0.7354
Epoch 8/20, Loss: 0.4899, Val AUC: 0.7350
Epoch 9/20, Loss: 0.4892, Val AUC: 0.7346
Epoch 10/20, Loss: 0.4889, Val AUC: 0.7342
Epoch 12/20, Loss: 0.4882, Val AUC: 0.7330


In [125]:
#딥러닝 앙상블상블

final_ensemble_dl_test = 0.4 * stacking_test_meta + 0.6 * stacking_test_meta_tab
final_ensemble_dl_submission = 0.4 * stacking_submission_meta + 0.6 * stacking_submission_meta_tab

###########################################
# 4. 최적 Threshold 찾기 및 평가
###########################################
precision, recall, thresholds = precision_recall_curve(y_test, final_ensemble_dl_test)
f1_scores = (2 * precision * recall) / (precision + recall + 1e-8)
optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f"Optimal Threshold: {optimal_threshold:.4f}")

final_roc_auc = roc_auc_score(y_test, final_ensemble_dl_test)
print(f"Final Ensemble ROC-AUC: {final_roc_auc:.4f}")

Y_test_pred = (final_ensemble_dl_test > optimal_threshold).astype(int)
print("\n=== Final Test Evaluation ===")
print(classification_report(y_test, Y_test_pred))


Optimal Threshold: 0.2549
Final Ensemble ROC-AUC: 0.7375

=== Final Test Evaluation ===
              precision    recall  f1-score   support

           0       0.89      0.55      0.68     38145
           1       0.38      0.79      0.51     13126

    accuracy                           0.61     51271
   macro avg       0.63      0.67      0.60     51271
weighted avg       0.76      0.61      0.64     51271



In [133]:
###########################################
# 3. 최종 앙상블: ML 앙상블 + TabNet (Soft Voting)
###########################################
# ML 앙상블과 TabNet 예측을 동일한 비율로 결합 (예: 50:50)
final_ensemble_pred_proba_test = 0.5 * ensemble_pred_proba_test + 0.5 * final_ensemble_dl_test.ravel()

# (submission 파일의 예측도 동일하게 처리해야 합니다. 예시로 ML 앙상블과 TabNet submission 예측값의 평균을 사용)
# 여기서는 간단히 ML 앙상블 예측값을 재사용합니다.
final_ensemble_pred_proba_submission = 0.5 * ensemble_pred_proba_submission + 0.5 * final_ensemble_dl_submission.ravel() # 실제 submission에서는 별도로 계산 필요

###########################################
# 4. 최적 Threshold 찾기 및 평가
###########################################
precision, recall, thresholds = precision_recall_curve(y_test, final_ensemble_pred_proba_test)
f1_scores = (2 * precision * recall) / (precision + recall + 1e-8)
optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f"Optimal Threshold: {optimal_threshold:.4f}")

final_roc_auc = roc_auc_score(y_test, final_ensemble_pred_proba_test)
print(f"Final Ensemble ROC-AUC: {final_roc_auc:.4f}")

Y_test_pred = (final_ensemble_pred_proba_test > optimal_threshold).astype(int)
print("\n=== Final Test Evaluation ===")
print(classification_report(y_test, Y_test_pred))


Optimal Threshold: 0.2817
Final Ensemble ROC-AUC: 0.7386

=== Final Test Evaluation ===
              precision    recall  f1-score   support

           0       0.89      0.55      0.68     38145
           1       0.38      0.80      0.51     13126

    accuracy                           0.61     51271
   macro avg       0.63      0.67      0.60     51271
weighted avg       0.76      0.61      0.64     51271



In [134]:
submission_filename = f'./data/submission_tabnet_focal_ml{final_roc_auc:.4f}.csv'
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['probability'] = final_ensemble_pred_proba_submission.ravel()
sample_submission.to_csv(submission_filename, index=False)
print(f"\n✅ Submission file saved: {submission_filename}")


✅ Submission file saved: ./data/submission_tabnet_focal_ml0.7386.csv
